# MScFE 620 DERIVATIVE PRICING - Group Work Project # 2

In [1]:
import numpy as np
import pandas as pd
from math import exp, sqrt

# -------- Assignment parameters (common) --------
S0 = 80.0
r = 0.055
sigma_ref = 0.35   # Provided general param; not used directly in Heston
T = 0.25          # 3 months
K = S0            # ATM

# -------- Heston parameters (given) --------
v0 = 0.032        # initial variance (3.2%)
kappa = 1.85
theta = 0.045

# -------- Model choice parameter (not in brief) --------
xi = 0.60         # vol-of-vol (assumption; adjust if needed)

# -------- Monte Carlo controls --------
n_paths = 150_000
n_steps = 63      # ~trading days in a quarter
seed = 42

## Question 5: Heston Model (Monte Carlo) for ATM European Call & Put with ρ = -0.30

In [2]:
rho = -0.30       # correlation for Question 5

def price_heston_euro_call_put_mc(
    S0, K, r, T, v0, kappa, theta, xi, rho, n_paths=150_000, n_steps=63, seed=42
):
    """
    Heston MC pricing (European call & put).
    Variance: full-truncation Euler; Stock: log-Euler under risk-neutral drift.
    Returns call_price, put_price, call_se, put_se, call_ci, put_ci.
    """
    rng = np.random.default_rng(seed)
    dt = T / n_steps
    sqrt_dt = sqrt(dt)
    one_minus_rho2 = sqrt(1.0 - rho**2)

    S = np.full(n_paths, S0, dtype=np.float64)
    v = np.full(n_paths, v0, dtype=np.float64)

    for _ in range(n_steps):
        z1 = rng.standard_normal(n_paths)
        z2i = rng.standard_normal(n_paths)
        z2 = rho * z1 + one_minus_rho2 * z2i

        v_pos = np.maximum(v, 0.0)
        # variance (CIR) update with full truncation
        v = v + kappa * (theta - v_pos) * dt + xi * np.sqrt(v_pos) * sqrt_dt * z2
        v = np.maximum(v, 0.0)

        # stock update (risk-neutral)
        S *= np.exp((r - 0.5 * v_pos) * dt + np.sqrt(v_pos * dt) * z1)

    disc = exp(-r * T)
    call = disc * np.maximum(S - K, 0.0)
    put  = disc * np.maximum(K - S, 0.0)

    def mean_se_ci(x):
        m = x.mean()
        se = x.std(ddof=1) / np.sqrt(len(x))
        ci = (m - 1.96 * se, m + 1.96 * se)
        return m, se, ci

    call_m, call_se, call_ci = mean_se_ci(call)
    put_m,  put_se,  put_ci  = mean_se_ci(put)

    return call_m, put_m, call_se, put_se, call_ci, put_ci

call_p, put_p, call_se, put_se, call_ci, put_ci = price_heston_euro_call_put_mc(
    S0, K, r, T, v0, kappa, theta, xi, rho, n_paths, n_steps, seed
)

df = pd.DataFrame({
    "Model": ["Heston (MC)", "Heston (MC)"],
    "Option": ["ATM Call", "ATM Put"],
    "S0": [S0, S0],
    "K": [K, K],
    "r": [r, r],
    "T (yrs)": [T, T],
    "v0": [v0, v0],
    "kappa": [kappa, kappa],
    "theta": [theta, theta],
    "xi": [xi, xi],
    "rho": [rho, rho],
    "Paths": [n_paths, n_paths],
    "Steps": [n_steps, n_steps],
    "Price": [round(call_p, 2), round(put_p, 2)],
    "Std Error": [call_se, put_se],
    "95% CI": [f"[{call_ci[0]:.3f}, {call_ci[1]:.3f}]",
                f"[{put_ci[0]:.3f}, {put_ci[1]:.3f}]"]
})

print(df.to_string(index=False), '\n')

print(f'Call Option Price with rho = -0.30: {df.iloc[0]["Price"]:.2f}')
print(f'Put Option Price with rho = -0.30: {df.iloc[1]["Price"]:.2f}')

# Optional quick sanity checks (no dividends):
discK = K * exp(-r*T)
parity_rhs = S0 - discK
print(f"\nParity check: C - P = {call_p - put_p:.6f}  vs  S0 - K*e^(-rT) = {parity_rhs:.6f}")
print(f"Bounds: Call >= {max(S0 - discK, 0):.4f}, Put <= {discK:.4f}")


      Model   Option   S0    K     r  T (yrs)    v0  kappa  theta  xi  rho  Paths  Steps  Price  Std Error         95% CI
Heston (MC) ATM Call 80.0 80.0 0.055     0.25 0.032   1.85  0.045 0.6 -0.3 150000     63   3.38   0.011842 [3.356, 3.403]
Heston (MC)  ATM Put 80.0 80.0 0.055     0.25 0.032   1.85  0.045 0.6 -0.3 150000     63   2.29   0.010966 [2.273, 2.316] 

Call Option Price with rho = -0.30: 3.38
Put Option Price with rho = -0.30: 2.29

Parity check: C - P = 1.084825  vs  S0 - K*e^(-rT) = 1.092472
Bounds: Call >= 1.0925, Put <= 78.9075


## Question 6: Heston Model (Monte Carlo) for ATM European Call & Put with ρ = -0.70

In [3]:
rho = -0.70       # correlation for Question 6

call_p, put_p, call_se, put_se, call_ci, put_ci = price_heston_euro_call_put_mc(
    S0, K, r, T, v0, kappa, theta, xi, rho, n_paths, n_steps, seed
)

df = pd.DataFrame({
    "Model": ["Heston (MC)", "Heston (MC)"],
    "Option": ["ATM Call", "ATM Put"],
    "S0": [S0, S0],
    "K": [K, K],
    "r": [r, r],
    "T (yrs)": [T, T],
    "v0": [v0, v0],
    "kappa": [kappa, kappa],
    "theta": [theta, theta],
    "xi": [xi, xi],
    "rho": [rho, rho],
    "Paths": [n_paths, n_paths],
    "Steps": [n_steps, n_steps],
    "Price": [round(call_p, 2), round(put_p, 2)],
    "Std Error": [call_se, put_se],
    "95% CI": [f"[{call_ci[0]:.3f}, {call_ci[1]:.3f}]",
                f"[{put_ci[0]:.3f}, {put_ci[1]:.3f}]"]
})

print(df.to_string(index=False), '\n')

print(f'Call Option Price with rho = -0.70: {df.iloc[0]["Price"]:.2f}')
print(f'Put Option Price with rho = -0.70: {df.iloc[1]["Price"]:.2f}')

# Optional quick sanity checks (no dividends):
discK = K * exp(-r*T)
parity_rhs = S0 - discK
print(f"\nParity check: C - P = {call_p - put_p:.6f}  vs  S0 - K*e^(-rT) = {parity_rhs:.6f}")
print(f"Bounds: Call >= {max(S0 - discK, 0):.4f}, Put <= {discK:.4f}")


      Model   Option   S0    K     r  T (yrs)    v0  kappa  theta  xi  rho  Paths  Steps  Price  Std Error         95% CI
Heston (MC) ATM Call 80.0 80.0 0.055     0.25 0.032   1.85  0.045 0.6 -0.7 150000     63   3.40   0.009755 [3.380, 3.418]
Heston (MC)  ATM Put 80.0 80.0 0.055     0.25 0.032   1.85  0.045 0.6 -0.7 150000     63   2.32   0.012081 [2.299, 2.346] 

Call Option Price with rho = -0.70: 3.40
Put Option Price with rho = -0.70: 2.32

Parity check: C - P = 1.076305  vs  S0 - K*e^(-rT) = 1.092472
Bounds: Call >= 1.0925, Put <= 78.9075


## Question 7: Greeks (Delta & Gamma) for Q5 and Q6 under Heston via MC central differences

In [4]:
import numpy as np
import pandas as pd
from math import exp, sqrt

# ---------- Common assignment parameters ----------
S0 = 80.0
K = 80.0
r = 0.055
T = 0.25          # 3 months
v0 = 0.032
kappa = 1.85
theta = 0.045
# Not specified in brief; documented assumption:
xi = 0.60         # vol-of-vol (can test sensitivity if desired)

# MC controls
n_paths = 150_000
n_steps = 63                  # ~trading days in a quarter
seed_base = 2025              # fixed so all bumps reuse CRNs
h = 0.50                      # price bump for central differences (Δ, Γ)

def heston_mc_price(S0, K, r, T, v0, kappa, theta, xi, rho, n_paths, n_steps, seed):
    """
    Monte Carlo price for European call & put under Heston.
    Variance: full-truncation Euler; Stock: log-Euler; CRNs via seed.
    Returns (call_price, put_price).
    """
    rng = np.random.default_rng(seed)
    dt = T / n_steps
    sqrt_dt = sqrt(dt)
    one_minus_rho2 = sqrt(max(1.0 - rho**2, 0.0))

    S = np.full(n_paths, S0, dtype=np.float64)
    v = np.full(n_paths, v0, dtype=np.float64)

    for _ in range(n_steps):
        z1 = rng.standard_normal(n_paths)
        z2i = rng.standard_normal(n_paths)
        z2 = rho * z1 + one_minus_rho2 * z2i

        v_pos = np.maximum(v, 0.0)
        # CIR (variance) with full truncation
        v = v + kappa * (theta - v_pos) * dt + xi * np.sqrt(v_pos) * sqrt_dt * z2
        v = np.maximum(v, 0.0)

        # Stock (risk-neutral) with log-Euler
        S *= np.exp((r - 0.5 * v_pos) * dt + np.sqrt(v_pos * dt) * z1)

    disc = exp(-r * T)
    call = disc * np.maximum(S - K, 0.0)
    put  = disc * np.maximum(K - S, 0.0)
    return call.mean(), put.mean()

def greeks_central_diff(S0, h, rho, seed_base):
    """
    Central differences with common random numbers:
      Δ ≈ [V(S+h) - V(S-h)] / (2h)
      Γ ≈ [V(S+h) - 2V(S) + V(S-h)] / h^2
    Returns dict with price, delta, gamma for call and put.
    """
    # Reuse identical RNG streams across bumps to reduce variance
    c_m, p_m = heston_mc_price(S0 - h, K, r, T, v0, kappa, theta, xi, rho, n_paths, n_steps, seed_base)
    c_0, p_0 = heston_mc_price(S0    , K, r, T, v0, kappa, theta, xi, rho, n_paths, n_steps, seed_base)
    c_p, p_p = heston_mc_price(S0 + h, K, r, T, v0, kappa, theta, xi, rho, n_paths, n_steps, seed_base)

    delta_c = (c_p - c_m) / (2*h)
    gamma_c = (c_p - 2*c_0 + c_m) / (h**2)

    delta_p = (p_p - p_m) / (2*h)
    gamma_p = (p_p - 2*p_0 + p_m) / (h**2)

    return {
        "Call Price": c_0, "Call Delta": delta_c, "Call Gamma": gamma_c,
        "Put Price":  p_0, "Put Delta":  delta_p, "Put Gamma":  gamma_p
    }

In [5]:
# Q5: rho = -0.30
out_30 = greeks_central_diff(S0, h, rho=-0.30, seed_base=seed_base)
# Q6: rho = -0.70
out_70 = greeks_central_diff(S0, h, rho=-0.70, seed_base=seed_base)

df = pd.DataFrame([
    {"Question": "Q5 (rho=-0.30)", "Option": "Call",
        "Price": round(out_30["Call Price"], 2),
        "Delta": out_30["Call Delta"], "Gamma": out_30["Call Gamma"]},
    {"Question": "Q5 (rho=-0.30)", "Option": "Put",
        "Price": round(out_30["Put Price"], 2),
        "Delta": out_30["Put Delta"], "Gamma": out_30["Put Gamma"]},
    {"Question": "Q6 (rho=-0.70)", "Option": "Call",
        "Price": round(out_70["Call Price"], 2),
        "Delta": out_70["Call Delta"], "Gamma": out_70["Call Gamma"]},
    {"Question": "Q6 (rho=-0.70)", "Option": "Put",
        "Price": round(out_70["Put Price"], 2),
        "Delta": out_70["Put Delta"], "Gamma": out_70["Put Gamma"]},
])

# Print nicely; keep raw decimals for Greeks
pd.set_option("display.float_format", lambda x: f"{x:.6f}")
print(df.to_string(index=False))

# Optional quick parity/bounds check on prices (no dividends)
discK = K * exp(-r*T)
parity_rhs = S0 - discK
print(f"\nParity target (S0 - K*e^(-rT)) = {parity_rhs:.6f}")


      Question Option    Price     Delta    Gamma
Q5 (rho=-0.30)   Call 3.360000  0.624258 0.058250
Q5 (rho=-0.30)    Put 2.300000 -0.375342 0.058250
Q6 (rho=-0.70)   Call 3.390000  0.671037 0.049600
Q6 (rho=-0.70)    Put 2.320000 -0.328601 0.049600

Parity target (S0 - K*e^(-rT)) = 1.092472


## Question 8: Merton Jump-Diffusion (Monte Carlo) for ATM European Call & Put (lambda = 0.75)

In [6]:
import numpy as np
import pandas as pd
from math import exp, sqrt, log

# ---------- Inputs ----------
S0 = 80.0
K  = 80.0
r  = 0.055
sigma = 0.35       # diffusion (non-jump) volatility
T  = 0.25          # 3 months
# Merton jump params
lmbda = 0.75       # jump intensity
muJ    = -0.5      # jump size mean (log jump, Normal)
deltaJ = 0.22      # jump size std

# MC controls
n_paths = 500_000
seed = 42

def merton_mc_prices(S0, K, r, sigma, T, lmbda, muJ, deltaJ, n_paths=200_000, seed=0):
    """
    Monte Carlo pricing under Merton jump-diffusion.
    Log-price at T: log S_T = log S_0 + (r - λk - 0.5σ^2)T + σ W_T + sum_{i=1}^{N_T} Y_i,
    where N_T ~ Poisson(λT), Y_i ~ Normal(muJ, deltaJ^2), k = E[e^Y - 1].
    Returns (call_price, put_price, call_se, put_se, call_ci, put_ci).
    """
    rng = np.random.default_rng(seed)
    # Jump compensator k = E[e^Y - 1]
    k = np.exp(muJ + 0.5 * (deltaJ**2)) - 1.0

    # Draw Poisson jump counts and Brownian
    N = rng.poisson(lmbda * T, size=n_paths)                 # number of jumps
    W = rng.normal(0.0, sqrt(T), size=n_paths)               # Brownian

    # Sum of jump log-sizes given N: Normal(N*muJ, N*deltaJ^2)
    Y_sum = rng.normal(N * muJ, np.sqrt(N * (deltaJ**2)))

    # Terminal log price
    log_ST = (log(S0)
              + (r - lmbda * k - 0.5 * sigma**2) * T
              + sigma * W
              + Y_sum)

    ST = np.exp(log_ST)

    disc = exp(-r * T)
    call_payoffs = np.maximum(ST - K, 0.0)
    put_payoffs  = np.maximum(K - ST, 0.0)

    call = disc * call_payoffs
    put  = disc * put_payoffs

    # Means
    call_price = call.mean()
    put_price  = put.mean()

    # Standard errors & 95% CIs
    call_se = call.std(ddof=1) / np.sqrt(n_paths)
    put_se  = put.std(ddof=1) / np.sqrt(n_paths)

    def ci(m, se): return (m - 1.96*se, m + 1.96*se)

    return call_price, put_price, call_se, put_se, ci(call_price, call_se), ci(put_price, put_se)

In [7]:
c, p, cse, pse, cci, pci = merton_mc_prices(S0, K, r, sigma, T, lmbda, muJ, deltaJ, n_paths, seed)

df = pd.DataFrame({
    "Model": ["Merton (MC)", "Merton (MC)"],
    "Option": ["ATM Call", "ATM Put"],
    "S0": [S0, S0],
    "K": [K, K],
    "r": [r, r],
    "T (yrs)": [T, T],
    "sigma": [sigma, sigma],
    "lambda": [lmbda, lmbda],
    "mu_J": [muJ, muJ],
    "delta_J": [deltaJ, deltaJ],
    "Paths": [n_paths, n_paths],
    "Price": [round(c, 2), round(p, 2)],
    "Std Error": [cse, pse],
    "95% CI": [f"[{cci[0]:.3f}, {cci[1]:.3f}]",
                f"[{pci[0]:.3f}, {pci[1]:.3f}]"]
})
pd.set_option("display.float_format", lambda x: f"{x:.6f}")
print(df.to_string(index=False), '\n')

print(f'Call Option Price: {df.iloc[0]["Price"]:.2f}')
print(f'Put Option Price: {df.iloc[1]["Price"]:.2f}')

# Quick parity/bounds check (no dividends)
discK = K * exp(-r*T)
print(f"\nParity target (S0 - K*e^(-rT)) = {S0 - discK:.6f}")
print(f"C - P = {c - p:.6f}   |   Bounds: Call >= {max(S0 - discK, 0):.4f},  Put <= {discK:.4f}")


      Model   Option        S0         K        r  T (yrs)    sigma   lambda      mu_J  delta_J  Paths    Price  Std Error         95% CI
Merton (MC) ATM Call 80.000000 80.000000 0.055000 0.250000 0.350000 0.750000 -0.500000 0.220000 500000 8.330000   0.016266 [8.299, 8.363]
Merton (MC)  ATM Put 80.000000 80.000000 0.055000 0.250000 0.350000 0.750000 -0.500000 0.220000 500000 7.170000   0.017338 [7.132, 7.200] 

Call Option Price: 8.33
Put Option Price: 7.17

Parity target (S0 - K*e^(-rT)) = 1.092472
C - P = 1.165077   |   Bounds: Call >= 1.0925,  Put <= 78.9075


## Question 9: Merton Jump-Diffusion (Monte Carlo) for ATM European Call & Put (lambda = 0.25)

In [8]:
import numpy as np
import pandas as pd
from math import exp, sqrt, log

# ---------- Inputs ----------
S0 = 80.0
K  = 80.0
r  = 0.055
sigma = 0.35          # diffusion (non-jump) volatility
T  = 0.25             # 3 months

# Merton jump parameters (Q9)
lmbda  = 0.25         # jump intensity
muJ    = -0.5         # mean of log jump size
deltaJ = 0.22         # std of log jump size

# Monte Carlo controls
n_paths = 500_000
seed = 42

def merton_mc_prices(S0, K, r, sigma, T, lmbda, muJ, deltaJ, n_paths=200_000, seed=0):
    """
    Monte Carlo pricing under Merton jump-diffusion.
    log S_T = log S_0 + (r - λk - 0.5σ^2)T + σ W_T + sum_{i=1}^{N_T} Y_i
      N_T ~ Poisson(λT), Y_i ~ Normal(muJ, deltaJ^2), k = E[e^Y - 1]
    Returns (call_price, put_price, call_se, put_se, call_ci, put_ci).
    """
    rng = np.random.default_rng(seed)

    # Jump compensator k = E[e^Y - 1]
    k = np.exp(muJ + 0.5 * (deltaJ**2)) - 1.0

    # Draw Poisson jump counts and Brownian term
    N = rng.poisson(lmbda * T, size=n_paths)              # number of jumps
    W = rng.normal(0.0, sqrt(T), size=n_paths)            # Brownian increment

    # Sum of log-jumps given N: Normal(N*muJ, N*deltaJ^2)
    Y_sum = rng.normal(N * muJ, np.sqrt(N * (deltaJ**2)))

    # Terminal log-price and price
    log_ST = (log(S0)
              + (r - lmbda * k - 0.5 * sigma**2) * T
              + sigma * W
              + Y_sum)
    ST = np.exp(log_ST)

    # Discounted payoffs
    disc = exp(-r * T)
    call = disc * np.maximum(ST - K, 0.0)
    put  = disc * np.maximum(K - ST, 0.0)

    # Means
    call_price = call.mean()
    put_price  = put.mean()

    # Standard errors & 95% CIs
    call_se = call.std(ddof=1) / np.sqrt(n_paths)
    put_se  = put.std(ddof=1) / np.sqrt(n_paths)

    def ci(m, se): return (m - 1.96*se, m + 1.96*se)

    return call_price, put_price, call_se, put_se, ci(call_price, call_se), ci(put_price, put_se)

In [9]:
c, p, cse, pse, cci, pci = merton_mc_prices(
    S0, K, r, sigma, T, lmbda, muJ, deltaJ, n_paths=n_paths, seed=seed
)

df = pd.DataFrame({
    "Model": ["Merton (MC)", "Merton (MC)"],
    "Option": ["ATM Call", "ATM Put"],
    "S0": [S0, S0],
    "K": [K, K],
    "r": [r, r],
    "T (yrs)": [T, T],
    "sigma": [sigma, sigma],
    "lambda": [lmbda, lmbda],
    "mu_J": [muJ, muJ],
    "delta_J": [deltaJ, deltaJ],
    "Paths": [n_paths, n_paths],
    "Price": [round(c, 2), round(p, 2)],
    "Std Error": [cse, pse],
    "95% CI": [f"[{cci[0]:.3f}, {cci[1]:.3f}]",
                f"[{pci[0]:.3f}, {pci[1]:.3f}]"]
})

pd.set_option("display.float_format", lambda x: f"{x:.6f}")
print(df.to_string(index=False), '\n')

print(f'Call Option Price: {df.iloc[0]["Price"]:.2f}')
print(f'Put Option Price: {df.iloc[1]["Price"]:.2f}')

# Quick parity & bounds checks (no dividends)
discK = K * exp(-r*T)
parity_target = S0 - discK
print(f"\nParity target (S0 - K*e^(-rT)) = {parity_target:.6f}")
print(f"C - P = {c - p:.6f}   |   Bounds: Call >= {max(S0 - discK, 0):.4f},  Put <= {discK:.4f}")


      Model   Option        S0         K        r  T (yrs)    sigma   lambda      mu_J  delta_J  Paths    Price  Std Error         95% CI
Merton (MC) ATM Call 80.000000 80.000000 0.055000 0.250000 0.350000 0.250000 -0.500000 0.220000 500000 6.830000   0.014292 [6.805, 6.861]
Merton (MC)  ATM Put 80.000000 80.000000 0.055000 0.250000 0.350000 0.250000 -0.500000 0.220000 500000 5.720000   0.013121 [5.699, 5.750] 

Call Option Price: 6.83
Put Option Price: 5.72

Parity target (S0 - K*e^(-rT)) = 1.092472
C - P = 1.108999   |   Bounds: Call >= 1.0925,  Put <= 78.9075


## Question 10: Greeks (Delta & Gamma) for Q8 and Q9 under Merton Jump-Diffusion

In [10]:
import numpy as np
import pandas as pd
from math import exp, sqrt, log

# ---------- Common inputs ----------
S0 = 80.0
K  = 80.0
r  = 0.055
sigma = 0.35
T  = 0.25

# ---------- Parameter sets ----------
# Q8: lambda=0.75 , Q9: lambda=0.25  (muJ, deltaJ same for both)
param_sets = [
    {"label": "Q8 (lambda=0.75)", "lmbda": 0.75, "muJ": -0.5, "deltaJ": 0.22},
    {"label": "Q9 (lambda=0.25)", "lmbda": 0.25, "muJ": -0.5, "deltaJ": 0.22},
]

# ---------- MC controls ----------
n_paths = 500_000           # increase to tighten noise
h = 0.50                    # S bump for central differences
seed = 42                 # fixed for reproducibility (CRNs)

rng_master = np.random.default_rng(seed)

def merton_draws(n_paths, T, lmbda, muJ, deltaJ, rng):
    """
    Generate common random numbers for one parameter set:
      - Poisson jump counts N ~ Poisson(lmbda*T)
      - Brownian W ~ N(0, sqrt(T))
      - Sum of log-jumps Y_sum ~ N(N*muJ, N*deltaJ^2)
    """
    N = rng.poisson(lmbda * T, size=n_paths)
    W = rng.normal(0.0, sqrt(T), size=n_paths)
    Y_sum = rng.normal(N * muJ, np.sqrt(N * (deltaJ**2)))
    return N, W, Y_sum

def merton_prices_from_draws(S0, K, r, sigma, T, lmbda, muJ, deltaJ, draws):
    """
    Price call/put using pre-generated draws (CRNs).
    log S_T = log S_0 + (r - λk - 0.5σ^2)T + σ W_T + sum Y_i
    """
    N, W, Y_sum = draws
    k = np.exp(muJ + 0.5 * (deltaJ**2)) - 1.0
    log_ST = (log(S0)
              + (r - lmbda * k - 0.5 * sigma**2) * T
              + sigma * W
              + Y_sum)
    ST = np.exp(log_ST)
    disc = exp(-r * T)
    call = disc * np.maximum(ST - K, 0.0)
    put  = disc * np.maximum(K - ST, 0.0)
    return call.mean(), put.mean()

def greeks_central_diff_merton(S0, h, params):
    """
    Central differences with CRNs:
      Δ ≈ [V(S+h) - V(S-h)] / (2h)
      Γ ≈ [V(S+h) - 2V(S) + V(S-h)] / h^2
    Returns dict with price, delta, gamma for call and put.
    """
    # Fresh RNG per parameter set for reproducible CRNs
    rng = np.random.default_rng(seed)
    draws = merton_draws(n_paths, T, params["lmbda"], params["muJ"], params["deltaJ"], rng)

    C_m, P_m = merton_prices_from_draws(S0 - h, K, r, sigma, T, params["lmbda"], params["muJ"], params["deltaJ"], draws)
    C_0, P_0 = merton_prices_from_draws(S0    , K, r, sigma, T, params["lmbda"], params["muJ"], params["deltaJ"], draws)
    C_p, P_p = merton_prices_from_draws(S0 + h, K, r, sigma, T, params["lmbda"], params["muJ"], params["deltaJ"], draws)

    dC = (C_p - C_m) / (2*h)
    gC = (C_p - 2*C_0 + C_m) / (h**2)

    dP = (P_p - P_m) / (2*h)
    gP = (P_p - 2*P_0 + P_m) / (h**2)

    return {
        "Call Price": round(C_0, 2),
        "Call Delta": dC,
        "Call Gamma": gC,
        "Put Price":  round(P_0, 2),
        "Put Delta":  dP,
        "Put Gamma":  gP,
    }

rows = []
for ps in param_sets:
    out = greeks_central_diff_merton(S0, h, ps)
    rows += [
        {"Question": ps["label"], "Option": "Call",
            "Price": out["Call Price"], "Delta": out["Call Delta"], "Gamma": out["Call Gamma"]},
        {"Question": ps["label"], "Option": "Put",
            "Price": out["Put Price"], "Delta": out["Put Delta"], "Gamma": out["Put Gamma"]},
    ]

df = pd.DataFrame(rows)
pd.set_option("display.float_format", lambda x: f"{x:.6f}")
print(df.to_string(index=False))


        Question Option    Price     Delta    Gamma
Q8 (lambda=0.75)   Call 8.330000  0.650004 0.022342
Q8 (lambda=0.75)    Put 7.170000 -0.350903 0.022342
Q9 (lambda=0.25)   Call 6.830000  0.598640 0.026484
Q9 (lambda=0.25)    Put 5.720000 -0.401566 0.026484


## Question 13: American Call under Heston (LSMC) + Delta/Gamma via central differences

In [11]:
import numpy as np
import pandas as pd
from math import exp, sqrt

# ---------- Assignment inputs ----------
S0, K = 80.0, 80.0
r, T  = 0.055, 0.25         # 3 months
v0, kappa, theta = 0.032, 1.85, 0.045
# Not specified in brief; documented assumption:
xi = 0.60                   # vol-of-vol
rhos = [-0.30, -0.70]       # Q13 repeats Q5 (rho=-0.30) & Q6 (rho=-0.70)

# ---------- MC / LSMC controls ----------
n_paths = 150_000
n_steps = 63                 # ~trading days in a quarter
dt = T / n_steps
h = 0.50                     # central bump for Greeks
seed = 2025                  # fixed to use common random numbers (CRNs)

# ---------- Heston path simulation using supplied noise (CRNs) ----------
def simulate_heston_paths_with_noise(S0, v0, n_paths, n_steps, r, kappa, theta, xi, rho, dt, z1_all, z2i_all):
    S = np.full((n_steps + 1, n_paths), S0, dtype=np.float64)
    v = np.full((n_steps + 1, n_paths), v0, dtype=np.float64)
    one_minus = np.sqrt(max(1.0 - rho**2, 0.0))
    sqrt_dt = np.sqrt(dt)

    for t in range(n_steps):
        z1 = z1_all[t]
        z2 = rho * z1 + one_minus * z2i_all[t]

        v_pos = np.maximum(v[t], 0.0)
        v_next = v[t] + kappa * (theta - v_pos) * dt + xi * np.sqrt(v_pos) * sqrt_dt * z2
        v_next = np.maximum(v_next, 0.0)

        S_next = S[t] * np.exp((r - 0.5 * v_pos) * dt + np.sqrt(v_pos * dt) * z1)

        v[t + 1] = v_next
        S[t + 1] = S_next

    return S, v

# ---------- Longstaff–Schwartz for American Call ----------
def lsm_american_call(S_paths, v_paths, r, dt, K):
    n_steps = S_paths.shape[0] - 1
    n_paths = S_paths.shape[1]
    disc = np.exp(-r * dt)

    # Cashflows and first exercise times
    CF = np.maximum(S_paths[-1] - K, 0.0)
    ex_time = np.full(n_paths, n_steps, dtype=int)

    # Backward induction
    for t in range(n_steps - 1, -1, -1):   # include t = 0
        CF *= disc
        S_t = S_paths[t]
        payoff = np.maximum(S_t - K, 0.0)

        alive = ex_time > t
        itm = payoff > 0
        mask = alive & itm
        if not np.any(mask):
            continue

        # Simple, effective polynomial basis with variance cross-term
        X = np.column_stack([
            np.ones(mask.sum()),
            S_t[mask], S_t[mask]**2, S_t[mask]**3,
            v_paths[t, mask],
            v_paths[t, mask] * S_t[mask]
        ])
        Y = CF[mask]  # continuation value one step ahead (already discounted)

        beta, *_ = np.linalg.lstsq(X, Y, rcond=None)
        C_hat = X @ beta

        exercise_now = payoff[mask] >= C_hat
        if np.any(exercise_now):
            idx = np.nonzero(mask)[0][exercise_now]
            CF[idx] = payoff[idx]
            ex_time[idx] = t

    return CF.mean()  # this is at t=0 since CF was discounted step-by-step

# ---------- European call (same draws) for comparison ----------
def european_call_from_paths(S_paths, r, T, K):
    disc = np.exp(-r * T)
    ST = S_paths[-1]
    return disc * np.maximum(ST - K, 0.0).mean()

# ---------- Driver for one rho: price + Greeks via central differences (CRNs) ----------
def american_call_price_and_greeks(S0, rho):
    # Common random numbers for S0-h, S0, S0+h to reduce finite-diff noise
    rng = np.random.default_rng(seed)
    z1 = rng.standard_normal((n_steps, n_paths))
    z2i = rng.standard_normal((n_steps, n_paths))

    # Base S0
    S0_paths, v_paths = simulate_heston_paths_with_noise(S0, v0, n_paths, n_steps, r, kappa, theta, xi, rho, dt, z1, z2i)
    C0_amer = lsm_american_call(S0_paths, v_paths, r, dt, K)
    C0_euro = european_call_from_paths(S0_paths, r, T, K)  # for commentary

    # Bumps (reuse same z's for CRNs)
    Sm_paths, vm_paths = simulate_heston_paths_with_noise(S0 - h, v0, n_paths, n_steps, r, kappa, theta, xi, rho, dt, z1, z2i)
    Sp_paths, vp_paths = simulate_heston_paths_with_noise(S0 + h, v0, n_paths, n_steps, r, kappa, theta, xi, rho, dt, z1, z2i)
    Cm = lsm_american_call(Sm_paths, vm_paths, r, dt, K)
    Cp = lsm_american_call(Sp_paths, vp_paths, r, dt, K)

    # Greeks (central differences)
    delta = (Cp - Cm) / (2 * h)
    gamma = (Cp - 2 * C0_amer + Cm) / (h ** 2)

    return C0_amer, C0_euro, delta, gamma

# ---------- Run for both rho values and print a tidy table ----------
rows = []
for rho in rhos:
    C_amer, C_euro, dC, gC = american_call_price_and_greeks(S0, rho)
    rows.append({
        "rho": rho,
        "American Call Price": round(C_amer, 2),   # submission requires cents
        "European Call (same draws)": round(C_euro, 2),
        "Early-Exercise Premium": round(C_amer - C_euro, 4),
        "Delta": dC,
        "Gamma": gC,
        "Paths": n_paths,
        "Steps": n_steps
    })

df = pd.DataFrame(rows)
pd.set_option("display.float_format", lambda x: f"{x:.6f}")
print(df.to_string(index=False))


      rho  American Call Price  European Call (same draws)  Early-Exercise Premium    Delta    Gamma  Paths  Steps
-0.300000             3.350000                    3.370000               -0.018300 0.631978 0.066013 150000     63
-0.700000             3.400000                    3.400000               -0.001000 0.673460 0.052519 150000     63


## Question 14: European Up-and-In Call Option (Heston Model)

In [12]:
import numpy as np
import pandas as pd
from math import exp, sqrt

# ---------- Heston / Contract Inputs (from Q6) ----------
S0    = 80.0
r     = 0.055
T     = 0.25            # 3 months
v0    = 0.032
kappa = 1.85
theta = 0.045
rho   = -0.70
xi    = 0.60            # not specified in brief; assumption

K = 95.0                # strike for Q14
B = 95.0                # up-and-in barrier

# ---------- Monte Carlo Controls ----------
n_paths = 200_000
n_steps = 63            # ~trading days in a quarter
seed    = 141
dt      = T / n_steps

def heston_paths_and_hits(S0, v0, r, T, kappa, theta, xi, rho, n_paths, n_steps, seed, B=None):
    """
    Simulate Heston paths with full-truncation Euler (variance) and log-Euler (price).
    If B is provided, returns an array 'hit' indicating whether the barrier was reached.
    Returns terminal prices S_T, hit_flags (or None if B is None).
    """
    rng = np.random.default_rng(seed)
    S = np.full(n_paths, S0, dtype=np.float64)
    v = np.full(n_paths, v0, dtype=np.float64)
    hit = np.zeros(n_paths, dtype=bool) if B is not None else None

    sqrt_dt = sqrt(dt)
    one_minus_rho2 = sqrt(max(1.0 - rho**2, 0.0))

    for _ in range(n_steps):
        z1 = rng.standard_normal(n_paths)
        z2i = rng.standard_normal(n_paths)
        z2 = rho * z1 + one_minus_rho2 * z2i

        v_pos = np.maximum(v, 0.0)
        # Variance update (CIR with full truncation)
        v = v + kappa * (theta - v_pos) * dt + xi * np.sqrt(v_pos) * sqrt_dt * z2
        v = np.maximum(v, 0.0)

        # Price update (risk-neutral)
        S *= np.exp((r - 0.5 * v_pos) * dt + np.sqrt(v_pos * dt) * z1)

        if B is not None:
            hit |= (S >= B)

    return S, hit

def price_vanilla_call(S_T, K, r, T):
    disc = exp(-r * T)
    pay = np.maximum(S_T - K, 0.0)
    price = disc * pay.mean()
    se    = disc * pay.std(ddof=1) / np.sqrt(len(S_T))
    return price, se

def price_up_in_call(S_T, hit, K, r, T):
    disc = exp(-r * T)
    pay = np.where(hit, np.maximum(S_T - K, 0.0), 0.0)
    price = disc * pay.mean()
    se    = disc * pay.std(ddof=1) / np.sqrt(len(S_T))
    return price, se

def ci95(mean, se):
    return (mean - 1.96 * se, mean + 1.96 * se)

if __name__ == "__main__":
    # Simulate paths once (same paths for both options)
    S_T, hit = heston_paths_and_hits(
        S0, v0, r, T, kappa, theta, xi, rho, n_paths, n_steps, seed, B=B
    )

    # Vanilla call (K=95) and Up-and-In call (B=95, K=95)
    c_van, se_van = price_vanilla_call(S_T, K, r, T)
    c_cui, se_cui = price_up_in_call(S_T, hit, K, r, T)

    ci_van = ci95(c_van, se_van)
    ci_cui = ci95(c_cui, se_cui)

    # Assemble comparison table (rounded prices to cents as per submission rules)
    df = pd.DataFrame({
        "Option":   ["Vanilla European Call", "Up-and-In Call (CUI)"],
        "Strike":   [K, K],
        "Barrier":  [np.nan, B],
        "rho":      [rho, rho],
        "xi":       [xi, xi],
        "Paths":    [n_paths, n_paths],
        "Steps":    [n_steps, n_steps],
        "Price":    [round(c_van, 2), round(c_cui, 2)],
        "Std Error":[se_van, se_cui],
        "95% CI":   [f"[{ci_van[0]:.3f}, {ci_van[1]:.3f}]",
                     f"[{ci_cui[0]:.3f}, {ci_cui[1]:.3f}]"]
    })

    pd.set_option("display.float_format", lambda x: f"{x:.6f}")
    print(df.to_string(index=False))

    # Optional diagnostics
    hit_rate = hit.mean()
    itm_rate_on_hit = (np.maximum(S_T - K, 0.0)[hit] > 0).mean() if hit_rate > 0 else 0.0
    print(f"\nBarrier hit rate: {hit_rate:.4%}")
    print(f"ITM finish given hit: {itm_rate_on_hit:.4%}")


               Option    Strike   Barrier       rho       xi  Paths  Steps    Price  Std Error         95% CI
Vanilla European Call 95.000000       NaN -0.700000 0.600000 200000     63 0.020000   0.000696 [0.018, 0.021]
 Up-and-In Call (CUI) 95.000000 95.000000 -0.700000 0.600000 200000     63 0.020000   0.000696 [0.018, 0.021]

Barrier hit rate: 1.2070%
ITM finish given hit: 66.9014%


## Question 15: European Down-and-In Put Option (Merton Model)

In [13]:
# Q15 — Merton Jump-Diffusion: European Down-and-In Put (PDI) vs Vanilla Put
# Uses Q8 parameters; barrier = 65, strike = 65. Discrete monitoring (63 steps).

import numpy as np
import pandas as pd
from math import exp, sqrt, log

# ---------- Base inputs (from Q8) ----------
S0    = 80.0
r     = 0.055
T     = 0.25
sigma = 0.35
# Merton jumps (Q8)
lmbda  = 0.75
muJ    = -0.5
deltaJ = 0.22

# ---------- Barrier / Strike for Q15 ----------
K = 65.0       # strike
B = 65.0       # down-and-in barrier

# ---------- Monte Carlo controls ----------
n_paths = 200_000
n_steps = 63                  # ~trading days in a quarter
seed    = 1015
dt      = T / n_steps

def simulate_merton_with_barrier(S0, r, sigma, T, lmbda, muJ, deltaJ,
                                 n_paths, n_steps, seed, B=None):
    """
    Time-stepping simulation under Merton jump-diffusion.
    Within each step dt:
      - Poisson jumps: N ~ Poisson(lambda*dt)
      - Sum of log-jumps: Y_sum ~ Normal(N*muJ, sqrt(N)*deltaJ)
      - Diffusive increment: sigma * sqrt(dt) * Z
      - Log update: d ln S = (r - lambda*k - 0.5*sigma^2)dt + sigma*sqrt(dt)Z + Y_sum
    Barrier (down) is monitored discretely: hit |= (S <= B).
    Returns terminal prices S_T and 'hit' flags (or None if B is None).
    """
    rng = np.random.default_rng(seed)
    # Jump compensator: k = E[e^Y - 1]
    k = np.exp(muJ + 0.5 * (deltaJ**2)) - 1.0

    logS = np.full(n_paths, log(S0), dtype=np.float64)
    hit  = np.zeros(n_paths, dtype=bool) if B is not None else None

    for _ in range(n_steps):
        # Jumps
        N = rng.poisson(lmbda * dt, size=n_paths)
        Y_sum = rng.normal(N * muJ, np.sqrt(N * (deltaJ**2)))
        # Diffusion
        Z = rng.normal(0.0, 1.0, size=n_paths)

        logS += (r - lmbda * k - 0.5 * sigma**2) * dt + sigma * sqrt(dt) * Z + Y_sum
        S = np.exp(logS)

        if B is not None:
            hit |= (S <= B)

    return np.exp(logS), hit

def price_put_vanilla(S_T, K, r, T):
    disc = exp(-r * T)
    pay = np.maximum(K - S_T, 0.0)
    price = disc * pay.mean()
    se    = disc * pay.std(ddof=1) / np.sqrt(len(S_T))
    return price, se

def price_put_down_in(S_T, hit, K, r, T):
    disc = exp(-r * T)
    pay = np.where(hit, np.maximum(K - S_T, 0.0), 0.0)
    price = disc * pay.mean()
    se    = disc * pay.std(ddof=1) / np.sqrt(len(S_T))
    return price, se

def ci95(mean, se):
    return (mean - 1.96 * se, mean + 1.96 * se)

In [15]:
# Simulate paths and barrier hits once (same paths for both options)
S_T, hit = simulate_merton_with_barrier(
    S0, r, sigma, T, lmbda, muJ, deltaJ,
    n_paths=n_paths, n_steps=n_steps, seed=seed, B=B
)

# Vanilla European put (K=65) and Down-and-In put (B=65, K=65)
p_van, se_van = price_put_vanilla(S_T, K, r, T)
p_pdi, se_pdi = price_put_down_in(S_T, hit, K, r, T)

ci_van = ci95(p_van, se_van)
ci_pdi = ci95(p_pdi, se_pdi)

# Assemble comparison table (round prices to cents per submission rules)
df = pd.DataFrame({
    "Option":      ["Vanilla European Put", "Down-and-In Put (PDI)"],
    "Strike":      [K, K],
    "Barrier":     [np.nan, B],
    "lambda":      [lmbda, lmbda],
    "mu_J":        [muJ, muJ],
    "delta_J":     [deltaJ, deltaJ],
    "Paths":       [n_paths, n_paths],
    "Steps":       [n_steps, n_steps],
    "Price":       [round(p_van, 2), round(p_pdi, 2)],
    "Std Error":   [se_van, se_pdi],
    "95% CI":      [f"[{ci_van[0]:.3f}, {ci_van[1]:.3f}]",
                    f"[{ci_pdi[0]:.3f}, {ci_pdi[1]:.3f}]"]
})

pd.set_option("display.float_format", lambda x: f"{x:.6f}")
print(df.to_string(index=False))

# Diagnostics: how often did the barrier get triggered?
hit_rate = hit.mean()
itm_rate = (np.maximum(K - S_T, 0.0) > 0).mean()
itm_given_hit = (np.maximum(K - S_T[hit], 0.0) > 0).mean() if hit_rate > 0 else 0.0
print(f"\nBarrier hit rate: {hit_rate:.2%}")
print(f"Finish ITM rate (vanilla): {itm_rate:.2%}")
print(f"Finish ITM given hit: {itm_given_hit:.2%}")


               Option    Strike   Barrier   lambda      mu_J  delta_J  Paths  Steps    Price  Std Error         95% CI
 Vanilla European Put 65.000000       NaN 0.750000 -0.500000 0.220000 200000     63 2.790000   0.017017 [2.752, 2.818]
Down-and-In Put (PDI) 65.000000 65.000000 0.750000 -0.500000 0.220000 200000     63 2.790000   0.017017 [2.752, 2.818]

Barrier hit rate: 25.49%
Finish ITM rate (vanilla): 18.55%
Finish ITM given hit: 72.75%
